### Metadata source

In [6]:
import sys
import os
import pandas as pd
import json
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_core.rate_limiters import InMemoryRateLimiter

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.articles import create_static_metadata, create_keywords_tags_fuzzy, keywords_dict
# from src.llm import get_gemini_llm_client
# from src.prompts import get_metadata_prompt

if not load_dotenv():
    raise Exception('Error loading .env file. Make sure to place a valid OPEN_AI_KEY in the .env file.')

In [2]:
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.5,  # <-- Gemini Free Tier
    check_every_n_seconds=0.1,
)

llm_client = get_gemini_llm_client(
    max_tokens=1024,
    temperature=0.2,
    rate_limiter=rate_limiter,
)

ValidationError: 1 validation error for ChatOpenAI
model_name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

Setup the paths to data sources

In [7]:
ARTICLES_CLEAN_DIR = os.path.join("..", "data", "articles_clean")
METADATA_PATH = os.path.join("..", "data", "metadata.csv")

Extract metadata

In [8]:
class ArticleTags(BaseModel):
    tags: list[str]

In [9]:
articles = os.listdir(ARTICLES_CLEAN_DIR)
import random
random.seed(42)
random.shuffle(articles)
# with open(, "r", encoding="utf-8") as file:
#     article = json.load(file)

article_ls = []
for name in articles[:10]:
    with open(os.path.join(ARTICLES_CLEAN_DIR, name), "r", encoding="utf-8") as file:
        article = json.load(file)

    article_ls.append(article)

article_df = pd.DataFrame(article_ls)

In [10]:
article_df

,id,published_at,author,title,category,ressort,text
0,948de0b1-b3b7-4c45-b22a-a074d3761cfc,2010-06-09 18:41,Walter Hämmerle,Unerbittliche Unvernunft,Leitartikel,Meinung,In Deutschland gibt es einige. Einer von ihnen...
1,d3da5c1b-9f10-4603-a648-d511cee9c14e,2016-02-17 17:50,WZ-Korrespondentin Martyna Czarnowska,Englisches Frühstück,Politik,Nachrichten,"Brüssel. ""Ein ""englisches Frühstück"": Das könn..."
2,a23cee8a-a2e3-43b7-b62b-6e5c3c0f2e87,2008-03-26 18:57,Helmut Dité,Gewinne sprudeln im Osten,Wirtschaft,Nachrichten,Im Gegenteil: Nachdem der Gewinn im CEE-Raum 2...
3,9fa0d59f-06ed-434f-9a7e-aabeff65fcbf,2015-12-17 18:15,Michael Schmölzer,Starke Ansage - leere Drohung?,Politik,Nachrichten,Wien/Berlin/Brüssel. Im Streit um die EU-weite...
4,dda1a4cf-76b9-41b0-a637-d0b7e12625d9,2022-11-14 09:30,Peter De Coensel,Wirtschaftspolitik als Waffe,Gastkommentare,Meinung,Von einer vorübergehenden Inflation im Zusamme...
5,0cf87a14-3d90-43f3-9659-c5e3f230add2,2019-09-18 11:00,Jan Michael Marchart,Das Jobmigranten-Karussell,Politik,Nachrichten,Wer es bis dahin nicht für möglich gehalten ha...
6,68990817-3939-4518-845c-5d7e5c0057a2,2017-06-07 17:49,Eva Stanzl,Die Bewahrung des Mythos,Wissen,Nachrichten,Ein Dutzend Journalisten inspiziert das hinter...
7,0cf4b22c-931e-4438-b7c1-620634b9bb01,2011-09-12 18:21,Stefan Melichar,Justiz ermittelt gegen Ex-Hypo-Berater,Wirtschaft,Nachrichten,Wien.\nDie - ohnehin schon ziemlich umfangreic...
8,de9c2c3e-4d94-4c79-92af-8d85a1f9a7cf,2007-04-17 17:48,WZ-Korrespondentin Alexandra Klausmann,Streik bei Skoda legt Produktion lahm,Wirtschaft,Nachrichten,"Ziel des Streiks, so Jaroslav Povsik, Vorsitze..."
9,d1c08103-21e1-4104-9c84-bcb776b0609c,2015-02-25 18:06,Brigitte Pechar,Grünes Misstrauen,Politik,Nachrichten,"Wien. Hypo-Untersuchungsausschuss, Islamgesetz..."


In [25]:
import pandas as pd

pd.Series([len(a["text"]) for a in article_df.iterrows()]).describe()
# around 4000 characters per article -> ~1000 tokens  (based on 4k random sample)

count     4000.000000
mean      3756.063250
std       2519.838299
min        196.000000
25%       2089.500000
50%       3093.000000
75%       4720.000000
max      23613.000000
dtype: float64

In [23]:
from nltk.corpus import stopwords
import string

stop_words = set(stopwords.words("english"))

# TODO
def clean_text(text):
    text = text.lower()
    text = "".join([char if char not in string.punctuation else " " for char in text])
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return " ".join(words)

In [24]:
article_df["text_cleaned"] = article_df["text"].apply(clean_text)

In [11]:
from transformers import pipeline
import multiprocessing as mp
import pandas as pd

# Example candidate labels
candidate_labels = ['financial crisis', 'sustainability', 'fake news', 'AI', 'digitalization', 'local journalism', 'covid', 'demographics', 'innovation', "other"]

# Initialize the pipeline (consider doing this within each process for GPU utilization)
def init_model():
    global classifier
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli") #, device=0)

# Worker function to process a chunk of articles
def process_chunk(chunk):
    results = []
    for article in chunk:
        # Optionally, pre-process article text (e.g., summarize or truncate)
        # For simplicity, we just truncate to 512 tokens here:
        # article = clean_text(article)
        truncated_article = " ".join(article.split()[:1025])
        res = classifier(truncated_article, candidate_labels, multi_label=False)
        results.append(res["labels"][0])  # Taking the top label
    return results

In [ ]:
# Assume you have your articles in a list or DataFrame column
# For demonstration, create a dummy DataFrame:

# Split the DataFrame into chunks for each worker (adjust chunk size as needed)
num_workers = 1 #mp.cpu_count()  # or set this to match the number of machines or cores you want to use
chunks = [article_df["text"].tolist()[i::num_workers] for i in range(num_workers)]

# Create a pool of workers and initialize the model in each process
with mp.Pool(processes=num_workers, initializer=init_model) as pool:
    results = pool.map(process_chunk, chunks)

# Flatten results and attach to the DataFrame
flat_results = [label for sublist in results for label in sublist]
article_df["predicted_topic"] = flat_results

# Save or further process your results
# df.to_csv("tagged_articles.csv", index=False)

In [36]:
sample_text = "Trotz offensichtlicher Probleme erklärten Finanzexperten Anfang der 2000er Jahre die Marktbedingungen für weitgehend optimal. Sie vertrauten auf mathematische Modelle, in denen ideale Bedingungen angenommen wurden, die es im wirklichen Leben nicht gibt. Zum Beispiel besagte die Effizienzmarkthypothese, dass Finanzmärkte stets für eine effiziente Preisbildung und damit eine Stabilisierung der Wirtschaft sorgten, indem hohe Risiken mit hohen Aufschlägen und niedrige Risiken mit geringen Aufschlägen bei Finanzprodukten versehen würden. Eine Sichtweise mit Fehlern, denn Risiken sind, anders als in der modernen Finanztheorie unterstellt, keinesfalls genau berechenbar. An ihren Modellen hielten die Expertinnen und Experten dennoch fest, beispielsweise auch nach dem Beinahe-Bankrott des Hedgefonds Long-Term Capital Management (LTCM) im Jahr 1998. Dabei handelt es sich hierbei gemäß den genannten Modellen um ein „Ereignis“, das, statistisch betrachtet, nur einmal alle drei Milliarden Jahre eintreten sollte. Und das Ereignis war beileibe kein Einzelfall."


# sample_text = "Was weiß ChatGPT über mich? Das interessierte den Norweger Arve Hjalmar Holmen, als er - wie viele andere auch - seinen Namen ins Suchfenster des KI-Chatbots eintippte. Das Ergebnis war erschreckend: ChatGPT stellte den Beschwerdeführer als Mörder seiner eigenen Kinder dar. Er habe zwei Kinder ermordet und versucht, seinen dritten Sohn zu töten. Die Geschichte war erfunden, ChatGPT mischte aber reale Elemente aus Holmens Leben dazu. Anzahl und Geschlecht der Kinder stimmten, ebenso der Name seiner Heimatstadt. „Die Tatsache, dass jemand diese Inhalte lesen und für wahr halten könnte, macht mir am meisten Angst“, erzählt Holmen."

# sample_text = article_df["text_cleaned"].tolist()[0]

In [ ]:
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# german model
classifier = pipeline("zero-shot-classification", model="joeddav/xlm-roberta-large-xnli")

classifier(sample_text, candidate_labels, multi_label=False, clean_up_tokenization_spaces=False)

config.json:   0%|          | 0.00/734 [00:00<?, ?B/s]

C:\Users\Felix\miniforge3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Felix\.cache\huggingface\hub\models--joeddav--xlm-roberta-large-xnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [6]:
def process_article(filename: str, with_tags: bool = True, tags_type: ('llm', 'keywords') = 'keywords'):
    """
    Process an article and return its metadata.
    :param filename: The name of the article file.
    :param with_tags: Whether to include tags in the metadata.
    :param tags_type: The type of tags to include in the metadata.
    :return: The metadata of the article.
    """    
    article_path = os.path.join(ARTICLES_CLEAN_DIR, filename)
    with open(article_path, "r", encoding="utf-8") as file:
        article = json.load(file)
    
    # Create static metadata for the article
    article_metadata = create_static_metadata(article, filename)
    
    if with_tags:
        if tags_type == 'llm':
            # Prepare the prompt for the LLM using the article's text
            tags_prompt = get_metadata_prompt()
            query = tags_prompt.format(article_text=article["text"])
            
            # Invoke the LLM with structured output to extract tags
            llm = llm_client.with_structured_output(ArticleTags)
            response = llm.invoke([query])
            tags = response.tags
        elif tags_type == 'keywords':
            keywords = keywords_dict()
            tags = create_keywords_tags_fuzzy(article["text"], keywords)
        else:
            raise ValueError(f"Invalid tags_type: {tags_type}. Must be ('llm', 'keywords').")
    else:
        tags = []
    
    article_metadata["tags"] = tags
    return article_metadata

In [7]:
# List all cleaned article files
articles = os.listdir(ARTICLES_CLEAN_DIR)
metadata = []

# Adjust the max_workers based on available resources (None is max)
with ThreadPoolExecutor(max_workers=None) as executor:
    futures = {executor.submit(process_article, filename): filename for filename in articles}
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            result = future.result()
            metadata.append(result)
        except Exception as e:
            print(f"Error processing file {futures[future]}: {e}")


df_metadata = pd.DataFrame(metadata)
df_metadata.to_csv(METADATA_PATH, index=False)

100%|██████████| 87754/87754 [1:19:56<00:00, 18.30it/s]  


In [8]:
# Articles length statistics
df_metadata["words_count"].describe()

count    87754.000000
mean       511.983670
std        351.851569
min          1.000000
25%        281.000000
50%        422.000000
75%        632.000000
max       5931.000000
Name: words_count, dtype: float64

Categories by WZ

In [9]:
df_metadata["category"].describe()

count       87754
unique         47
top       Politik
freq        31919
Name: category, dtype: object

In [10]:
df_metadata["category"].value_counts()

category
Politik                                  31919
Wirtschaft                               16512
Kommentare                               11408
Gastkommentare                            7253
Wissen                                    5632
Europaarchiv                              5341
Leitartikel                               3292
Analysen                                  2478
Reflexionen                               2308
Recht                                      652
Auf Justitias Spuren                       196
Leserforum                                 170
Klimawandel                                 52
Wiener Zeitung - seit 1703                  47
Sterbehilfe                                 44
1914                                        39
100 Jahre Republik                          38
Stadtentwicklung                            28
Wald                                        27
100 Jahre Verfassung                        26
EU für mich                                 26
Asyl

In [11]:
# Tags statistics (only available if with_tags=True)
df_metadata.explode("tags")["tags"].value_counts()

tags
other               73976
COVID                5415
Fake News            4610
Digitalization       1149
Demographics          990
Innovation            823
Financial Crises      612
Sustainability        138
AI                     37
Local Journalism        4
Name: count, dtype: int64

In [12]:
# Missing authors
df_metadata["author"].isnull().sum()

np.int64(77)

Metadata df

In [13]:
df_metadata.head(5)

,id,title,author,published_at,words_count,filename,category,section,tags
0,cbbd50ec-c07b-4fcf-b7bf-dd0b7bacc887,100.000 Plätze mehr in zehn Jahren,Alexandra Grass,2003-10-08 00:00,264,100000-platze-mehr-in-zehn-jahren.json,Politik,Nachrichten,other
1,a0799ad0-f4ed-4989-8c9a-afa6e88677c6,1.700 Lehrlinge ohne Ausbildung,Werner Grotte,2004-10-06 00:00,255,1700-lehrlinge-ohne-ausbildung.json,Wirtschaft,Nachrichten,other
2,9866b0a2-9c97-4a49-b5c3-d6803a3f9eac,14 Gemeinden erheben schwere Vorwürfe gegen Ra...,Kid Möchel,2011-06-17 18:28,530,14-gemeinden-erheben-schwere-vorwurfe-gegen-ra...,Wirtschaft,Nachrichten,other
3,1e9ee2dd-f1cc-42c3-a61b-47cd64dd3fca,10.000 syrische Babys - geboren in einem ander...,Maysoon Mohammad Khalaf Al-Hijazat,2017-08-16 13:23,522,10000-syrische-babys-geboren-in-einem-anderen-...,Gastkommentare,Meinung,other
4,4fe39ec3-426f-4608-9def-225229a4a476,1700 Euro steuerfrei: Was der SPÖ-Plan bringt,Karl Ettinger,2019-08-26 18:14,824,1700-euro-steuerfrei-was-der-spo-plan-bringt.json,Politik,Nachrichten,other
